<a href="https://colab.research.google.com/github/Rodri-rf/Algorithm-design-and-analysis/blob/main/CSC_373_EC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC 373 EC 2 - DP
In this notebook you'll implement some DP algorithms that we studied in lecture. There are details on how to submit your work later in the notebook. Your screenshots should be submitted with HW2.

Something to watch out for - in lecture most things are 1-indexed but in Python things are 0-indexed.

Good luck!

In [1]:
# @title Run this cell to set up some helper functions

import networkx as nx
import graphviz
import random
import itertools
import timeit
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import numpy as np
from dataclasses import dataclass
from itertools import chain, combinations


pio.templates.default = "plotly_white"

from PIL import Image
from typing import Tuple

Town = dict[str, Tuple[int, int]]
Region = list[Town]

hoenn : Region =  {
    "Littleroot": (44,107)
    , "Oldale": (44, 91)
    , "Petalburg": (19, 91)
    , "Rustboro": (12, 62)
    , "Verdanturf": (44, 67)
    , "Mauville": (80, 66)
    , "Slateport": (75, 103)
    , "Lavaridge": (52, 43)
    , "Fallarbor": (35, 19)
    , "Fortree": (108, 19)
    , "Lilycove": (163,43)
    , "Mossdeep": (208, 59)
    , "Sootopolis": (180, 75)
    , "EverGrande": (228, 87)
    , "Dewford": (27, 131)
    , "Pacificdog": (148, 99)
}

def random_region():
    region = dict()
    for k in hoenn.keys():
        region[k] = (random.randint(30, 230), random.randint(30,138))
    return region


def unzip_region(region: Region):
    names = list(region.keys())
    xs, ys = zip(*(region.values()))
    return names, xs, ys


! wget https://archives.bulbagarden.net/media/upload/1/1f/Hoenn_RSE_Map.png

HOENN_ARRAY = np.asarray(Image.open("Hoenn_RSE_Map.png"))
HOENN_ARRAY_TRANSLUCENT = HOENN_ARRAY.copy()
HOENN_ARRAY_TRANSLUCENT[:,:,3] = 125
BLANK = np.ones(HOENN_ARRAY.shape)

def add_edge(fig, x1, y1, x2, y2, w):
  fig.add_trace(
    go.Scatter(x=[x1, x2], y=[y1,y2], mode="lines", line={"width":1, "color":"black"})
  )
  fig.add_annotation(x=(x1 + x2)/2, y=(y1 + y2)/2,
            text=str(int(w)), # round down to int so the plot is readable
            showarrow=False,
            xshift=16,
            yshift=16,
            font={"color":"black", "size":16}
  )
  fig.update_layout(showlegend=False)
  return fig

def plot_graph(g, region, no_bg=False):
    if no_bg:
        fig = px.imshow(np.zeros(HOENN_ARRAY.shape))
    else:
        fig = px.imshow(HOENN_ARRAY)
    names, xs, ys = unzip_region(region)
    fig.add_trace(
        go.Scatter(x=xs, y=ys, mode="markers", marker={"size":12, "color":"black"}, hovertext=names, name="")
    )
    fig.update_layout(showlegend=False, height=600, width=700)
    for u, v in g.edges():
        x1, y1 = region[u]
        x2, y2 = region[v]
        add_edge(fig, x1, y1, x2, y2, g.edges[(u, v)]['weight'])
    return fig

def euclidean_distance(x1, y1, x2, y2):
  return (abs(x1-x2)**2 + abs(y1-y2)**2)**0.5

def get_graph(region):
    G = nx.Graph()
    G.add_nodes_from(region.keys())
    edge_list = []
    for (k1, (x1, y1)) in region.items():
        for (k2, (x2, y2)) in region.items():
            if k1!=k2:
                weight = euclidean_distance(x1,y1,x2,y2)
                edge_list.append((k1, k2, weight))
    G.add_weighted_edges_from(edge_list)
    return G

def plot_cycle(g, region, C, no_bg=False):
    edges = zip(C, C[1:])
    cycle = g.edge_subgraph(edges)
    f = plot_graph(cycle, region, no_bg)
    print(sum(x["weight"] for _,_,x in cycle.edges.data(True)))
    return f

def time(function, input, number):
    return timeit.timeit(lambda: function(*input), number=number)


def plot_weighted_intervals_with_colors(intervals, colors):
    fig = go.Figure()
    for i in range(len(intervals)):
        s = intervals[i].start
        f = intervals[i].end
        w = intervals[i].weight
        color = colors[i]
        fig.add_trace(go.Scatter(
            x=[s, (s + f)/2, f],
            y=[i, i, i],
            mode="lines+text",
            text=["", str(w), ""],
            line=dict(color=color,width=5),
            textposition="top center"
        ))
    fig.update_layout(
        showlegend=False,
        height=450
    )
    return fig

def plot_selected(intervals, selected):
    colors = ["green" if i in selected else "black"  for i in range(len(intervals))]
    return plot_weighted_intervals_with_colors(intervals, colors)

def plot_intervals(intervals):
    colors = ["black"] * len(intervals)
    return plot_weighted_intervals_with_colors(intervals, colors)

sort_earliest_finish = lambda intervals : sorted(intervals, key=lambda y: y.end)

@dataclass
class Interval:
    start: float
    end: float
    weight: float
def get_random_weighted_intervals(n):
    return [
        Interval(*sorted([random.uniform(0, 1), random.uniform(0, 1)]), random.randint(1,100)) for _ in range(n)
    ]

@dataclass
class Item:
    v: int
    w: int

def random_items(n):
    return [
        Item(random.randint(1, 10), random.randint(1,10)) for _ in range(n)
    ]

def random_items_large_weight(n):
    return [
        Item(random.randint(1, 100), random.randint(1,100000)) for _ in range(n)
    ]

def conflicts(x, y):
    return y.start <= x.start < y.end or y.start <= x.end < y.end or x.start <= y.start < x.end or x.start <= y.end < x.end

def compatible(x, y):
    return not conflicts(x, y)


--2024-03-01 02:08:50--  https://archives.bulbagarden.net/media/upload/1/1f/Hoenn_RSE_Map.png
Resolving archives.bulbagarden.net (archives.bulbagarden.net)... 172.64.192.10, 172.64.193.10, 2606:4700:e6::ac40:c10a, ...
Connecting to archives.bulbagarden.net (archives.bulbagarden.net)|172.64.192.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8409 (8.2K) [image/png]
Saving to: ‘Hoenn_RSE_Map.png’

Hoenn_RSE_Map.png   100%[===================>]   8.21K  --.-KB/s    in 0s      

2024-03-01 02:08:50 (62.3 MB/s) - ‘Hoenn_RSE_Map.png’ saved [8409/8409]



# Practice: Weighted Interval Scheduling
(Not for credit)

In [ ]:
@dataclass
class Interval:
    start: float
    end: float
    weight: float

intervals = get_random_weighted_intervals(10)
plot_intervals(intervals)

In [ ]:
intervals = sort_earliest_finish(intervals)
plot_intervals(intervals)

You can plot selected intervals in green using the following function - in this example I'm selecting the first and second intervals to highlight. You can use this function to test your code later!

In [ ]:
plot_selected(intervals, [1,2])

Optional practice: Implement the weighted interval scheduling DP algorithm. Start by implementing an algorithm that only gives the optimal value.

In [ ]:
def get_p(intervals):
  pass

In [ ]:
def weighted_interval_scheduling_top_down(intervals):
  pass

def weighted_interval_scheduling_bottom_up(intervals):
  pass

In [ ]:
print(weighted_interval_scheduling_top_down(intervals))
print(weighted_interval_scheduling_bottom_up(intervals))

Now, extend the previous to return the optimal solution as well. You should return a tuple `(val, selected)` where `val` is the optimal value and `selected` is a list of indices of the intervals attaining the the optimal value.


In [ ]:
def weighted_interval_scheduling_top_down_return_opt(intervals):
  pass


def weighted_interval_scheduling_bottom_up_return_opt(intervals):
  pass



# TSP

In this section, we'll be solving the TSP problem in the Hoenn region using the DP described in class.

In [ ]:
hoenn

We've provided a helper function `plot_graph` that takes a `nx.Graph`, and a location dictionary and plots the graph. G is empty in the cell below, so the function displays a graph with no edges.

In [ ]:
G = nx.Graph()
plot_graph(G, hoenn)

To make our lives simpler - let's ignore all the roads and oceans, and imagine we are a bird (a Taillow if you will). That is, we can get to any city from any other city by flying a straight line between the two cities. Below, we have added these lines along with their corresponding distances.

In [ ]:
G = get_graph(hoenn) # fills in all edges
plot_graph(G, hoenn)

Solving the TSP problem on this graph corresponds to finding the fastest way to visit all the cities in Hoenn and returning to your starting city.

Here's a brute force solution. Note that the first line of the solution truncates the list of cities to consider (otherwise, it would be too slow).

In [ ]:
def brute_force_tsp(G):
    nodes = list(G.nodes())[:11] # TOGGLE THIS
    start = nodes[0]
    def cost(perm):
        edges = zip(perm, perm[1:])
        return sum( G.edges[e]["weight"]  for e in edges )

    return min(
        map(
            lambda x: [start] + list(x) + [start],
            itertools.permutations(nodes[1:])
        )
        , key = cost
    )

C = brute_force_tsp(G)
print(C)

Once you have a cycle, you can plot it like below:

In [ ]:
plot_cycle(G, hoenn, C)

Notice how some cities were excluded from the cycle because of the truncation step. If you played around with the truncation (i.e. perhaps changed 11 to 15), then you might see that this brute force solution will crashes instance. Let's see if a DP approach can do better...

## Task

Implement the DP algorithm for TSP from lecture. Your function should return both the optimal value and the cycle corresponding to the optimal value.

In [ ]:
# You might find the following function useful - it returns the set of all subsets of a given iterable...
def powerset(iterable):
  # Reference https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    xs = list(iterable)
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

def dp_tsp(G):
  pass


In [ ]:
val, C = dp_tsp(G)
C

In [ ]:
plot_cycle(G, hoenn, C)

Run the next cell to shuffle the locations of the cities. Run your TSP algorithm on the shuffled city and plot the it!

In [ ]:
random.seed(a="replace with your groups UTorIDs", version=2)
my_region = random_region()
plot_graph(nx.Graph(), my_region, no_bg=True)

In [ ]:
G = get_graph(my_region)
val, C = dp_tsp(G)
plot_cycle(G, my_region, C, no_bg=True)

## How to submit
Upload screenshots of the previous two cells (the one that generates a new region, and the one that prints the optimal hamiltonian path).

# Knapsack   

In [ ]:
@dataclass
class Item:
    v: int
    w: int

items = random_items(10)
capacity = 25
items

Here's a brute force implementation for the Knapsack problem that considers all possible subset of items.

In [ ]:


def knapsack_brute_force(items, capacity):
    opt = max(
        powerset(items), key =
        lambda x: -1 if sum(y.w for y in x) > capacity else sum(y.v for y in x)
    )
    val = sum(y.v for y in opt)
    return val, opt

knapsack_brute_force(items, capacity)

## Task

Implement the DP algorithm for the knapsack problem (the one that runs in time $O(nW)$). You can choose which implementation to use (bottom-up or top-down)

In [ ]:
def knapsack_return_opt(items, capacity):
  pass

knapsack_return_opt(items, capacity)

What happens when the capacity of your backpack is large?

In [ ]:
random.seed(a="replace with your groups UTorIDs", version=2)
N = 100
itemsL = random_items_large_weight(N)
capacityL = int((N * 50000)*0.7)
print(capacityL)

Warning: The next cell should crash!

In [ ]:
#print(knapsack_return_opt(itemsL, capacityL))

As we noted in lecture - there is another DP that works better when the total value of all the items under consideration are small. In this example, here is how the capacity of the backpack compares to the weight.

In [ ]:
print("Capacity", capacityL)
print("Total Value", sum(x.v for x in items))

Implement the other DP - the one that runs in time $O(nV)$.

In [ ]:
def knapsack_return_opt_small_values(items, capacity):
  pass


In [ ]:
print(knapsack_return_opt_small_values(itemsL, capacityL)[0])

## How to submit
Upload the following
* a screenshot of the cell that was used to generate itemsL (including the seed your team used).
* a screenshot of the cell above this one. (the one that prints the optimal value)